# Resume Screening
Version 1.0

***
## Pipeline

1. Resume Parsing  
    * The first subtask is parsing the resume, i.e., extracting information in a structured format from the document. 
2. Resume Matching
    * The second sub-task is extracting semantic information and actually understanding the underlying information.
3. Database Formation

***

## Import Libraries

In [ ]:
# !pip install PyMuPDF
# !pip install docx2pdf

In [27]:
import numpy
import sys
import fitz
from docx2pdf import convert
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
#nltk.download('punkt')
#nltk.download('stopwords')

## Reading the file

In [ ]:
def read_file(fname):
    if fname.split(".")[1] == 'docx':
        convert(fname)
        fname = fname.split(".")[0] + ".pdf"
    elif fname.split(".")[1] == 'pdf':
        pass
    else:
        print("Only PDF and docx types are supported!")
        return
    
    doc = fitz.open(fname)
    text = ""
    for page in doc:
        text = text + str(page.get_text())

    tx = " ".join(text.split('\n'))  
    #print(text)
    return text

In [ ]:
resume = read_file('Ahmed Osama-Long Version.docx')
resume

## Pre-processing
* It consists of some main steps
    * Lowercase
    * Removing Punctuation
    * Tokenization
    * Stopword Filtering
    * Stemming
    * Lemmatization

### Lowercase

In [ ]:
resume_lower = resume.lower()
resume_lower

### Removing Punctuation

In [ ]:
#print(string.punctuation)
## Note + needs to be kept -> C++
## Note @ needs to be kept -> May remove emails 
## Remove bullet points
## Remove Qoutes
resume_punc = "".join([char for char in resume_lower if char not in string.punctuation + "●•"])
print(resume_punc)

### Tokenization

In [ ]:
resume_tokens = nltk.word_tokenize(resume_punc)
resume_tokens

### Stopword Filtering

In [ ]:
stop_words = stopwords.words('english')
#print(stop_words)
resume_filtered_words = [word for word in resume_tokens if word not in stop_words]
#print(resume_filtered_words)
resume_filtered_words

### Stemming

In [ ]:
porter = PorterStemmer()
stemmed_resume = [porter.stem(word) for word in resume_filtered_words]
print(stemmed_resume)

## Feature Extraction

### Words Count (Frequency)

In [ ]:
resume_freq = dict()
for word in stemmed_resume:
    resume_freq[word] = resume_freq.get(word,0)+1
print(resume_freq)

# V2.0

## Reading the Data

In [5]:
import pandas as pd
import numpy as np

In [40]:
resume_df = pd.read_csv("Datasets/resume/Resume.csv")
resume_df.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [41]:
resume_df.describe()

,ID
count,2.484000e+03
mean,3.182616e+07
std,2.145735e+07
min,3.547447e+06
25%,1.754430e+07
50%,2.521031e+07
75%,3.611444e+07
max,9.980612e+07


## Preprocessing

### Cleaning data

In [42]:
##resume_df = resume_df[np.isfinite(pd.to_numeric(resume_df.ID, errors="coerce"))]
resume_df[['ID']] = resume_df[['ID']].apply(pd.to_numeric, errors='coerce')

In [43]:
resume_df.drop(columns=['Resume_html'], inplace = True)

### Lowercase

In [44]:
def to_lower(df):
    df["Resume_str"] = df["Resume_str"].str.lower()
    df["Category"] = df["Category"].str.lower()
    return df

In [46]:
lowered_df = to_lower(resume_df)
lowered_df.head()

,ID,Resume_str,Category
0,16852973,hr administrator/marketing associate\...,hr
1,22323967,"hr specialist, us hr operations ...",hr
2,33176873,hr director summary over 2...,hr
3,27018550,hr specialist summary dedica...,hr
4,17812897,hr manager skill highlights ...,hr


### Punctuation

In [47]:
def rem_punct(df):
    #" ".join([char for char in resume_lower if char not in string.punctuation + "●•"])
    df['punct_sent'] = df.apply(lambda row: "".join([char for char in row['Resume_str'] if char not in string.punctuation]), axis=1)
    return df

In [48]:
punct_df = rem_punct(lowered_df)
punct_df.head()

,ID,Resume_str,Category,punct_sent
0,16852973,hr administrator/marketing associate\...,hr,hr administratormarketing associate\n...
1,22323967,"hr specialist, us hr operations ...",hr,hr specialist us hr operations ...
2,33176873,hr director summary over 2...,hr,hr director summary over 2...
3,27018550,hr specialist summary dedica...,hr,hr specialist summary dedica...
4,17812897,hr manager skill highlights ...,hr,hr manager skill highlights ...


### Tokenization

In [49]:
def to_tokens(df):
    df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row['punct_sent']), axis=1)
    return df

In [50]:
tokenized_df = to_tokens(lowered_df)
tokenized_df.head()

,ID,Resume_str,Category,punct_sent,tokenized_sents
0,16852973,hr administrator/marketing associate\...,hr,hr administratormarketing associate\n...,"[hr, administratormarketing, associate, hr, ad..."
1,22323967,"hr specialist, us hr operations ...",hr,hr specialist us hr operations ...,"[hr, specialist, us, hr, operations, summary, ..."
2,33176873,hr director summary over 2...,hr,hr director summary over 2...,"[hr, director, summary, over, 20, years, exper..."
3,27018550,hr specialist summary dedica...,hr,hr specialist summary dedica...,"[hr, specialist, summary, dedicated, driven, a..."
4,17812897,hr manager skill highlights ...,hr,hr manager skill highlights ...,"[hr, manager, skill, highlights, hr, skills, h..."


### Stop Words

In [51]:
def rem_stop_words(df):
    stop_words = stopwords.words('english')
    df['stop_words'] = df.apply(lambda row: [word for word in row['tokenized_sents'] if word not in stop_words], axis=1)
    return df

In [52]:
stop_words_df = rem_stop_words(tokenized_df)
stop_words_df.head()

,ID,Resume_str,Category,punct_sent,tokenized_sents,stop_words
0,16852973,hr administrator/marketing associate\...,hr,hr administratormarketing associate\n...,"[hr, administratormarketing, associate, hr, ad...","[hr, administratormarketing, associate, hr, ad..."
1,22323967,"hr specialist, us hr operations ...",hr,hr specialist us hr operations ...,"[hr, specialist, us, hr, operations, summary, ...","[hr, specialist, us, hr, operations, summary, ..."
2,33176873,hr director summary over 2...,hr,hr director summary over 2...,"[hr, director, summary, over, 20, years, exper...","[hr, director, summary, 20, years, experience,..."
3,27018550,hr specialist summary dedica...,hr,hr specialist summary dedica...,"[hr, specialist, summary, dedicated, driven, a...","[hr, specialist, summary, dedicated, driven, d..."
4,17812897,hr manager skill highlights ...,hr,hr manager skill highlights ...,"[hr, manager, skill, highlights, hr, skills, h...","[hr, manager, skill, highlights, hr, skills, h..."


In [53]:
stop_words_df

,ID,Resume_str,Category,punct_sent,tokenized_sents,stop_words
0,16852973,hr administrator/marketing associate\...,hr,hr administratormarketing associate\n...,"[hr, administratormarketing, associate, hr, ad...","[hr, administratormarketing, associate, hr, ad..."
1,22323967,"hr specialist, us hr operations ...",hr,hr specialist us hr operations ...,"[hr, specialist, us, hr, operations, summary, ...","[hr, specialist, us, hr, operations, summary, ..."
2,33176873,hr director summary over 2...,hr,hr director summary over 2...,"[hr, director, summary, over, 20, years, exper...","[hr, director, summary, 20, years, experience,..."
3,27018550,hr specialist summary dedica...,hr,hr specialist summary dedica...,"[hr, specialist, summary, dedicated, driven, a...","[hr, specialist, summary, dedicated, driven, d..."
4,17812897,hr manager skill highlights ...,hr,hr manager skill highlights ...,"[hr, manager, skill, highlights, hr, skills, h...","[hr, manager, skill, highlights, hr, skills, h..."
...,...,...,...,...,...,...
2479,99416532,rank: sgt/e-5 non- commissioned offic...,aviation,rank sgte5 non commissioned officer i...,"[rank, sgte5, non, commissioned, officer, in, ...","[rank, sgte5, non, commissioned, officer, char..."
2480,24589765,"government relations, communications ...",aviation,government relations communications a...,"[government, relations, communications, and, o...","[government, relations, communications, organi..."
2481,31605080,geek squad agent professional...,aviation,geek squad agent professional...,"[geek, squad, agent, professional, profile, it...","[geek, squad, agent, professional, profile, su..."
2482,21190805,program director / office manager ...,aviation,program director office manager ...,"[program, director, office, manager, summary, ...","[program, director, office, manager, summary, ..."


### TF in document

In [72]:
from collections import Counter

In [73]:
def TF_doc(df):
    df['TF_doc'] = df.apply(lambda row: Counter(row['stop_words']), axis=1)
    return df

In [74]:
TF_doc_df = TF_doc(stop_words_df)
TF_doc_df

,ID,Resume_str,Category,punct_sent,tokenized_sents,stop_words,TF_doc
0,16852973,hr administrator/marketing associate\...,hr,hr administratormarketing associate\n...,"[hr, administratormarketing, associate, hr, ad...","[hr, administratormarketing, associate, hr, ad...","{'hr': 4, 'administratormarketing': 2, 'associ..."
1,22323967,"hr specialist, us hr operations ...",hr,hr specialist us hr operations ...,"[hr, specialist, us, hr, operations, summary, ...","[hr, specialist, us, hr, operations, summary, ...","{'hr': 7, 'specialist': 3, 'us': 3, 'operation..."
2,33176873,hr director summary over 2...,hr,hr director summary over 2...,"[hr, director, summary, over, 20, years, exper...","[hr, director, summary, 20, years, experience,...","{'hr': 8, 'director': 3, 'summary': 1, '20': 1..."
3,27018550,hr specialist summary dedica...,hr,hr specialist summary dedica...,"[hr, specialist, summary, dedicated, driven, a...","[hr, specialist, summary, dedicated, driven, d...","{'hr': 3, 'specialist': 3, 'summary': 1, 'dedi..."
4,17812897,hr manager skill highlights ...,hr,hr manager skill highlights ...,"[hr, manager, skill, highlights, hr, skills, h...","[hr, manager, skill, highlights, hr, skills, h...","{'hr': 32, 'manager': 3, 'skill': 1, 'highligh..."
...,...,...,...,...,...,...,...
2479,99416532,rank: sgt/e-5 non- commissioned offic...,aviation,rank sgte5 non commissioned officer i...,"[rank, sgte5, non, commissioned, officer, in, ...","[rank, sgte5, non, commissioned, officer, char...","{'rank': 5, 'sgte5': 2, 'non': 2, 'commissione..."
2480,24589765,"government relations, communications ...",aviation,government relations communications a...,"[government, relations, communications, and, o...","[government, relations, communications, organi...","{'government': 7, 'relations': 7, 'communicati..."
2481,31605080,geek squad agent professional...,aviation,geek squad agent professional...,"[geek, squad, agent, professional, profile, it...","[geek, squad, agent, professional, profile, su...","{'geek': 2, 'squad': 2, 'agent': 2, 'professio..."
2482,21190805,program director / office manager ...,aviation,program director office manager ...,"[program, director, office, manager, summary, ...","[program, director, office, manager, summary, ...","{'program': 4, 'director': 6, 'office': 4, 'ma..."


In [90]:
#for item in TF_doc_df.at[1, 'TF_doc']:
for item, value in TF_doc_df.at[50, 'TF_doc'].items():
    print(item, value)

hr 5
assistantpayroll 3
highlights 1
paychex 1
mastertax 1
abra 2
time 2
star 1
infotronics 1
store 1
master 1
dominion 1
quickbooks 2
ceridian 2
peoplesoft 2
sap 2
microsoft 2
word 2
excel 2
powerpoint 2
crystal 2
reports 3
experience 2
company 11
name 10
－ 11
city 11
state 17
human 9
resources 9
professional 1
offering 1
15 1
years 1
combined 1
payroll 21
manufacturing 2
academic 2
settings 1
accurate 5
timely 3
preparation 1
employee 13
tax 9
information 16
benefits 10
administration 2
highly 1
effective 1
employer 1
representation 1
unemployment 11
claims 7
hearings 3
012014 2
coordinator 2
process 13
weekly 11
100 2
hourly 3
salaried 3
associates 3
garnishment 3
levy 3
child 3
support 3
processing 3
compile 3
labor 3
hours 3
enter 4
new 7
hire 6
hrispayroll 5
system 7
assist 7
annual 9
open 4
enrollment 4
status 4
changes 4
personnel 4
benefit 6
records 5
respond 5
establish 3
written 4
policies 4
procedures 3
conflict 4
resolution 4
workers 5
compensation 5
accident 4
reporting 9

### TF in a Category

In [ ]:
# Most freququent element each document
for index, row in TF_doc_df.iterrows():
  for item, value in row["TF_doc"].items():
    print(item, value)
    break
  #print(row["TF_doc"])

In [54]:
stop_words_df.Category.unique()

array(['hr', 'designer', 'information-technology', 'teacher', 'advocate',
       'business-development', 'healthcare', 'fitness', 'agriculture',
       'bpo', 'sales', 'consultant', 'digital-media', 'automobile',
       'chef', 'finance', 'apparel', 'engineering', 'accountant',
       'construction', 'public-relations', 'banking', 'arts', 'aviation'],
      dtype=object)

In [122]:
TF_cat = pd.DataFrame(
                  index=pd.Index( stop_words_df.Category.unique()),
                  columns=pd.Index([]))
TF_cat

""
hr
designer
information-technology
teacher
advocate
business-development
healthcare
fitness
agriculture
bpo


In [125]:
# Most Frequent Words only
for index, row in TF_doc_df.iterrows():
  for item, value in row["TF_doc"].items():
    if item not in TF_cat.columns:
      TF_cat[item] = 0
      TF_cat.at[row["Category"], item] = value
    else:      
      TF_cat.at[row["Category"], item] += value

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """



In [126]:
TF_cat

,hr,senior,summary,sr,directv,assistant,regional,field,territory,general,...,veneer,cirque,soleils,dardiers,hance,productawareness,ciety,twinbrook,innocence,adultyouth
hr,1827,107,95,23,4,126,49,37,18,85,...,0,0,0,0,0,0,0,0,0,0
designer,3,86,91,13,0,47,3,53,3,28,...,0,0,0,0,0,0,0,0,0,0
information-technology,10,100,75,3,0,47,14,44,1,37,...,0,0,0,0,0,0,0,0,0,0
teacher,3,14,86,0,0,110,12,56,0,47,...,0,0,0,0,0,0,0,0,0,0
advocate,30,69,100,7,0,85,13,39,5,60,...,0,0,0,0,0,0,0,0,0,0
business-development,16,115,89,14,1,29,52,26,108,49,...,0,0,0,0,0,0,0,0,0,0
healthcare,36,73,86,6,0,114,50,45,40,66,...,0,0,0,0,0,0,0,0,0,0
fitness,24,50,89,1,0,96,16,30,17,115,...,0,0,0,0,0,0,0,0,0,0
agriculture,3,35,53,3,0,55,18,66,3,42,...,0,0,0,0,0,0,0,0,0,0
bpo,19,22,17,19,1,13,3,5,0,16,...,0,0,0,0,0,0,0,0,0,0


In [128]:
TF_final_df = TF_cat.T
TF_final_df.head(10)

,hr,designer,information-technology,teacher,advocate,business-development,healthcare,fitness,agriculture,bpo,...,chef,finance,apparel,engineering,accountant,construction,public-relations,banking,arts,aviation
hr,1827,3,10,3,30,16,36,24,3,19,...,11,17,17,11,40,7,16,26,30,6
senior,107,86,100,14,69,115,73,50,35,22,...,37,196,65,62,156,105,95,152,31,63
summary,95,91,75,86,100,89,86,89,53,17,...,96,107,75,90,116,106,91,96,75,97
sr,23,13,3,0,7,14,6,1,3,19,...,0,19,39,11,4,10,5,28,1,1
directv,4,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,2,0,0,0
assistant,126,47,47,110,85,29,114,96,55,13,...,53,59,144,39,59,62,118,91,100,61
regional,49,3,14,12,13,52,50,16,18,3,...,14,29,11,8,11,20,33,64,18,8
field,37,53,44,56,39,26,45,30,66,5,...,17,22,18,88,21,142,24,17,43,49
territory,18,3,1,0,5,108,40,17,3,0,...,0,1,8,0,2,0,6,23,3,4
general,85,28,37,47,60,49,66,115,42,16,...,68,208,57,32,402,168,47,63,50,71


In [130]:
TF_final_df.to_csv("train.csv")

In [133]:
for col in TF_final_df:
    print(TF_final_df[col]/TF_final_df[col].sum())
    break

hr                  0.025471
senior              0.001492
summary             0.001324
sr                  0.000321
directv             0.000056
                      ...   
productawareness    0.000000
ciety               0.000000
twinbrook           0.000000
innocence           0.000000
adultyouth          0.000000
Name: hr, Length: 54746, dtype: float64


In [134]:
TF_Normalized = TF_final_df.copy()
TF_Normalized

,hr,designer,information-technology,teacher,advocate,business-development,healthcare,fitness,agriculture,bpo,...,chef,finance,apparel,engineering,accountant,construction,public-relations,banking,arts,aviation
hr,1827,3,10,3,30,16,36,24,3,19,...,11,17,17,11,40,7,16,26,30,6
senior,107,86,100,14,69,115,73,50,35,22,...,37,196,65,62,156,105,95,152,31,63
summary,95,91,75,86,100,89,86,89,53,17,...,96,107,75,90,116,106,91,96,75,97
sr,23,13,3,0,7,14,6,1,3,19,...,0,19,39,11,4,10,5,28,1,1
directv,4,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
productawareness,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ciety,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
twinbrook,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
innocence,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [135]:
for col in TF_Normalized:
    TF_Normalized[col]/=TF_Normalized[col].sum()

In [136]:
TF_final_df

,hr,designer,information-technology,teacher,advocate,business-development,healthcare,fitness,agriculture,bpo,...,chef,finance,apparel,engineering,accountant,construction,public-relations,banking,arts,aviation
hr,0.025471,0.000051,0.000118,0.000056,0.000399,0.000224,0.000467,0.000376,0.000074,0.001182,...,0.000158,0.000236,0.000297,0.000144,0.000556,0.000097,0.000214,0.000369,0.000502,0.000085
senior,0.001492,0.001454,0.001178,0.000259,0.000918,0.001613,0.000948,0.000782,0.000866,0.001368,...,0.000532,0.002718,0.001137,0.000812,0.002170,0.001456,0.001270,0.002158,0.000519,0.000896
summary,0.001324,0.001538,0.000884,0.001593,0.001330,0.001248,0.001117,0.001393,0.001311,0.001057,...,0.001381,0.001484,0.001312,0.001179,0.001614,0.001470,0.001216,0.001363,0.001256,0.001379
sr,0.000321,0.000220,0.000035,0.000000,0.000093,0.000196,0.000078,0.000016,0.000074,0.001182,...,0.000000,0.000263,0.000682,0.000144,0.000056,0.000139,0.000067,0.000397,0.000017,0.000014
directv,0.000056,0.000000,0.000000,0.000000,0.000000,0.000014,0.000000,0.000000,0.000000,0.000062,...,0.000000,0.000000,0.000017,0.000000,0.000000,0.000000,0.000027,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
productawareness,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000014
ciety,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000028
twinbrook,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000028
innocence,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000028


### IDF

In [137]:
IDF_DF = TF_final_df.copy()
IDF_DF

,hr,designer,information-technology,teacher,advocate,business-development,healthcare,fitness,agriculture,bpo,...,chef,finance,apparel,engineering,accountant,construction,public-relations,banking,arts,aviation
hr,1827,3,10,3,30,16,36,24,3,19,...,11,17,17,11,40,7,16,26,30,6
senior,107,86,100,14,69,115,73,50,35,22,...,37,196,65,62,156,105,95,152,31,63
summary,95,91,75,86,100,89,86,89,53,17,...,96,107,75,90,116,106,91,96,75,97
sr,23,13,3,0,7,14,6,1,3,19,...,0,19,39,11,4,10,5,28,1,1
directv,4,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
productawareness,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ciety,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
twinbrook,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
innocence,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [143]:
#IDF_DF.drop(columns = ['IDF'], inplace = True)

In [145]:

IDF_DF['IDF'] = IDF_DF.apply(lambda row: 1+ np.log(24/row.astype(bool).sum()), axis=1)
IDF_DF

,hr,designer,information-technology,teacher,advocate,business-development,healthcare,fitness,agriculture,bpo,...,finance,apparel,engineering,accountant,construction,public-relations,banking,arts,aviation,IDF
hr,1827,3,10,3,30,16,36,24,3,19,...,17,17,11,40,7,16,26,30,6,1.000000
senior,107,86,100,14,69,115,73,50,35,22,...,196,65,62,156,105,95,152,31,63,1.000000
summary,95,91,75,86,100,89,86,89,53,17,...,107,75,90,116,106,91,96,75,97,1.000000
sr,23,13,3,0,7,14,6,1,3,19,...,19,39,11,4,10,5,28,1,1,1.133531
directv,4,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,2,0,0,0,2.232144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
productawareness,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.178054
ciety,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,4.178054
twinbrook,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,4.178054
innocence,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,4.178054


In [146]:
IDF_DF = IDF_DF[['IDF']]
IDF_DF

,IDF
hr,1.000000
senior,1.000000
summary,1.000000
sr,1.133531
directv,2.232144
...,...
productawareness,4.178054
ciety,4.178054
twinbrook,4.178054
innocence,4.178054


### TF-IDF

In [149]:
TF_IDF_DF = TF_final_df.copy()
TF_IDF_DF = TF_IDF_DF.multiply(IDF_DF["IDF"], axis="index")
TF_IDF_DF

,hr,designer,information-technology,teacher,advocate,business-development,healthcare,fitness,agriculture,bpo,...,chef,finance,apparel,engineering,accountant,construction,public-relations,banking,arts,aviation
hr,0.025471,0.000051,0.000118,0.000056,0.000399,0.000224,0.000467,0.000376,0.000074,0.001182,...,0.000158,0.000236,0.000297,0.000144,0.000556,0.000097,0.000214,0.000369,0.000502,0.000085
senior,0.001492,0.001454,0.001178,0.000259,0.000918,0.001613,0.000948,0.000782,0.000866,0.001368,...,0.000532,0.002718,0.001137,0.000812,0.002170,0.001456,0.001270,0.002158,0.000519,0.000896
summary,0.001324,0.001538,0.000884,0.001593,0.001330,0.001248,0.001117,0.001393,0.001311,0.001057,...,0.001381,0.001484,0.001312,0.001179,0.001614,0.001470,0.001216,0.001363,0.001256,0.001379
sr,0.000363,0.000249,0.000040,0.000000,0.000106,0.000223,0.000088,0.000018,0.000084,0.001340,...,0.000000,0.000299,0.000774,0.000163,0.000063,0.000157,0.000076,0.000451,0.000019,0.000016
directv,0.000124,0.000000,0.000000,0.000000,0.000000,0.000031,0.000000,0.000000,0.000000,0.000139,...,0.000000,0.000000,0.000039,0.000000,0.000000,0.000000,0.000060,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
productawareness,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000059
ciety,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000119
twinbrook,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000119
innocence,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000119


In [ ]:
TF_IDF_DF.to_csv("TF-IDF.csv")

In [101]:
#TF_cat["yes"] = np.nan
#TF_cat.at['hr', 'yes'] = 6
#TF_cat

,yes
hr,6.0
designer,NaN
information-technology,NaN
teacher,NaN
advocate,NaN
business-development,NaN
healthcare,NaN
fitness,NaN
agriculture,NaN
bpo,NaN


In [103]:
#flag = TF_cat.isnull()
#flag.at['arts','yes']

True